# Binary classification benchmark

This notebook compares binary classification models from various libraries. The dataset used is the [Higgs dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS).

In [2]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,skgarden,sklearn,tensorflow,torch --datename

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Sun Oct 06 2019 

CPython 3.7.4
IPython 7.4.0

creme 0.3.0
keras 2.2.4
skgarden 0.1.2
sklearn 0.21.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.2.10-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [16]:
from creme import compose
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import neighbors
from creme import optim
from creme import preprocessing
from creme import stats
from creme import stream
from creme import tree
from keras import layers
from keras import models
from keras import optimizers
import numpy as np
import skgarden
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [17]:
n_features = 28

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(), loss='binary_crossentropy')

names = [
    'is_signal',
    'lepton pT', 'lepton eta','lepton phi',
    'missing energy magnitude', 'missing energy phi', 'jet 1 pt',
    'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag',
    'jet 2 pt', 'jet 2 eta', 'jet 2 phi',
    'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta',
    'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt',
    'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag',
    'm_jj', 'm_jjj', 'm_lv', 'm_jlv',
    'm_bb', 'm_wbb', 'm_wwbb'
]
converters = {name: float for name in names}
converters['is_signal'] = lambda x: bool(float(x))


import itertools

n = 100_000 # 11_000_000

get_X_y = lambda: itertools.islice(stream.iter_csv('HIGGS.csv.gz', names=names, target_name='is_signal', converters=converters), n)

results = benchmark(
    get_X_y=get_X_y,
    n=n,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('creme', 'LogisticRegression', linear_model.LogisticRegression()),
        ('creme', 'PAClassifier', linear_model.PAClassifier()),
        ('creme', 'DecisionTreeClassifier', tree.DecisionTreeClassifier()),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasBinaryClassifier(keras_model)),
        ('PyTorch (CPU)', 'Linear', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=0.01)
        )),
        ('scikit-learn', 'SGDClassifier', ScikitLearnClassifier(
            sk_linear_model.SGDClassifier(loss='log'),
            classes=[False, True]
        )),
        ('scikit-learn', 'PassiveAggressiveClassifier', ScikitLearnClassifier(
            sk_linear_model.PassiveAggressiveClassifier(),
            classes=[False, True]
        )),
        ('scikit-garden', 'MondrianTreeClassifier', ScikitLearnClassifier(
            skgarden.MondrianTreeClassifier(random_state=42),
            classes=np.array([False, True])
        )),
        ('scikit-garden', 'MondrianForestClassifier', ScikitLearnClassifier(
            skgarden.MondrianForestClassifier(random_state=42),
            classes=np.array([False, True])
        ))
    ],
    get_metric=metrics.Accuracy
)

W1006 21:40:48.473205 139676324894528 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1006 21:40:48.483894 139676324894528 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [18]:
results

,Library,Model,Accuracy,Fit time,Average fit time,Predict time,Average predict time
0,creme,LogisticRegression,0.61810,"2s, 651ms, 809μs, 53ns","26μs, 518ns","1s, 179ms, 271μs, 21ns","11μs, 793ns"
1,creme,PAClassifier,0.55009,"3s, 715ms, 139μs, 617ns","37μs, 151ns","2s, 361ms, 426μs, 113ns","23μs, 614ns"
2,creme,DecisionTreeClassifier,0.64663,"39s, 148ms, 904μs, 801ns","391μs, 489ns","1s, 698ms, 218μs, 36ns","16μs, 982ns"
3,Keras on Tensorflow (CPU),Dense,0.61840,"44s, 389ms, 39μs, 941ns","443μs, 890ns","50s, 460ms, 240μs, 948ns","504μs, 602ns"
4,PyTorch (CPU),Linear,0.61840,"23s, 573ms, 157μs, 809ns","235μs, 732ns","14s, 303ms, 631μs, 398ns","143μs, 36ns"
5,scikit-learn,SGDClassifier,0.56161,"28s, 326ms, 176μs, 85ns","283μs, 262ns","7s, 952ms, 237μs, 187ns","79μs, 522ns"
6,scikit-learn,PassiveAggressiveClassifier,0.55009,"28s, 439ms, 591μs, 588ns","284μs, 396ns","8s, 266ms, 966μs, 821ns","82μs, 670ns"
7,scikit-garden,MondrianTreeClassifier,0.53875,"1m, 1s, 206ms, 916μs, 758ns","612μs, 69ns","15s, 303ms, 194μs, 953ns","153μs, 32ns"
8,scikit-garden,MondrianForestClassifier,0.60061,"11m, 58s, 515ms, 475μs, 454ns","7ms, 185μs, 155ns","1m, 29s, 362ms, 415μs, 438ns","893μs, 624ns"
